In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re

# 1. Caricamento Dati
df = pd.read_csv('IOC_DATASET.csv')
sns.set_theme(style="whitegrid") # Stile pulito per report accademici

# --- METRICA 1: Volume per Gruppo ---
plt.figure(figsize=(10, 6))
# Prendiamo i top 10 gruppi per leggibilità
top_groups = df['chat_name'].value_counts().head(10)
sns.barplot(x=top_groups.values, y=top_groups.index, palette="viridis")
plt.title("Volume di IoC estratti per Gruppo")
plt.xlabel("Numero di IoC")
plt.tight_layout()
plt.show()

# --- METRICA 2: Heatmap (Chi scambia cosa) ---
# Filtriamo per i top 5 gruppi e top 5 tipi di IoC per chiarezza
top_groups_list = df['chat_name'].value_counts().head(5).index
top_types_list = df['ioc_type'].value_counts().head(5).index
filtered_df = df[df['chat_name'].isin(top_groups_list) & df['ioc_type'].isin(top_types_list)]

# Creiamo la tabella pivot
crosstab = pd.crosstab(filtered_df['chat_name'], filtered_df['ioc_type'])

plt.figure(figsize=(12, 8))
sns.heatmap(crosstab, annot=True, fmt='d', cmap="YlGnBu", linewidths=.5)
plt.title("Specializzazione dei Gruppi: Quali IoC scambiano?")
plt.show()

# --- METRICA 3: Analisi del Contesto (Topic Modeling semplice) ---
def analyze_context(ioc_type):
    # Prende tutti i testi che contengono quel tipo di IoC
    texts = df[df['ioc_type'] == ioc_type]['text'].dropna().tolist()
    text_blob = " ".join(texts).lower()
    # Trova parole di almeno 4 lettere
    words = re.findall(r'\b\w{4,}\b', text_blob)
    # Rimuovi parole comuni (stopwords)
    stopwords = {'http', 'https', 'with', 'this', 'that', 'from', 'code', 'data'}
    clean_words = [w for w in words if w not in stopwords]
    return Counter(clean_words).most_common(5)

print("Parole chiave per IP:", analyze_context('ip4'))
print("Parole chiave per URL:", analyze_context('url'))


# --- METRICA 4: ANALISI PER TOPIC ---

# A. Volume totale per Topic (Chi ne ha di più?)
plt.figure(figsize=(8, 5))
topic_counts = df['topic'].value_counts()
sns.barplot(x=topic_counts.index, y=topic_counts.values, palette="rocket")
plt.title("Volume totale di IoC per Topic")
plt.ylabel("Numero di IoC")
plt.xlabel("Topic")
plt.tight_layout()
plt.savefig('metric_topic_volume.png')
plt.show()

# B. Heatmap Normalizzata: Topic vs Tipo IoC
# Normalizziamo (percentuale) per vedere la DISTRIBUZIONE, non solo il volume
crosstab_topic = pd.crosstab(df['topic'], df['ioc_type'], normalize='index')

# Selezioniamo solo i tipi di IoC più rilevanti per pulire il grafico
relevant_types = ['url', 'ip4', 'fqdn', 'email', 'phoneNumber', 'bitcoin', 'md5']
crosstab_topic_filtered = crosstab_topic[relevant_types]

plt.figure(figsize=(10, 6))
sns.heatmap(crosstab_topic_filtered, annot=True, fmt=".1%", cmap="Purples", linewidths=.5)
plt.title("Composizione percentuale degli IoC per Topic")
plt.ylabel("Topic")
plt.xlabel("Tipo di IoC")
plt.tight_layout()
plt.savefig('metric_topic_heatmap.png')
plt.show()

# C. Top 3 IoC più frequenti per ogni Topic
# Raggruppa per Topic e IoC, conta le occorrenze
top_ioc_by_topic = df.groupby(['topic', 'ioc_value']).size().reset_index(name='count')
# Ordina per topic e poi per conteggio decrescente
top_ioc_by_topic = top_ioc_by_topic.sort_values(['topic', 'count'], ascending=[True, False])
# Prendi i primi 3 per ogni gruppo
top3 = top_ioc_by_topic.groupby('topic').head(3)

print("\n=== TOP 3 IOC PIÙ SCAMBIATI PER TOPIC ===")
# Formattiamo l'output per renderlo leggibile
for topic, group in top3.groupby('topic'):
    print(f"\n📂 {topic.upper()}:")
    for _, row in group.iterrows():
        print(f"   - {row['ioc_value']} (Visto {row['count']} volte)")